In [2]:
import pandas as pd

# Work on syllogism form

In [5]:
df_form = pd.read_csv("./data/form_data.csv")

In [18]:
absurde = ["AO3","AO4","OA3","OA4"]
positif = ["AA1","AA2","AA4","AI2","AI4","IA1","IA4"]
universel = ["AA1","AA2","AA4","AE1","AE2","AE3","AE4","EA1","EA2","EA3","EA4"]
particulier = ["AI2","AI4","AO3","AO4","EI1","EI2","EI3","EI4","IA1","IA4","IE1","IE2","IE3","IE4","OA3","OA4"]

In [19]:
df_form_valide = df_form.loc[df_form["has_conclusion"]==True,:]

In [21]:
df_form_valide["mode"] = df_form_valide.task_form.apply(lambda x: "universel" if x in universel else "particulier")
df_form_valide["signe"] = df_form_valide.task_form.apply(lambda x: "positif" if x in positif else "negatif")
df_form_valide["absurde"] = df_form_valide.task_form.apply(lambda x: True if x in absurde else False)

/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_76729/1893931228.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_form_valide["mode"] = df_form_valide.task_form.apply(lambda x: "universel" if x in universel else "particulier")
/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_76729/1893931228.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_form_valide["signe"] = df_form_valide.task_form.apply(lambda x: "positif" if x in positif else "negatif")
/var/folders/py/4rt338cj1ks4z33

In [23]:
df_form_valide = df_form_valide.drop("Unnamed: 0", axis=1)

In [27]:
df_form_valide.to_csv("./data/form_data_final.csv", index=False)

# Work on nlp_resul

In [29]:
df_nlp_result = pd.read_csv("./data/nlp_result.csv")
df_idseq = pd.read_csv("./data/idseq_sentence.csv")

In [74]:
df_nlp_result["task_form"].value_counts()

IE4    109
II3    109
EO3    107
EO1    107
AI3    107
      ... 
AO4     96
OA3     95
IA4     94
IO4     93
IA1     93
Name: task_form, Length: 64, dtype: int64

In [42]:
df_nlp_result.shape

(6506, 7)

In [43]:
df_idseq.shape

(6506, 2)

In [35]:
df_nlp_result.head()

,Unnamed: 0,id_seq,task_form,best_bert_00_index,bert_00success,best_mnli_00_index,mnli_00success
0,0,1_0_R,IA4,2,1,2,1
1,1,1_1_R,EA3,0,0,7,1
2,2,1_2_R,AI2,3,1,2,1
3,3,1_3_R,AA1,0,1,4,0
4,4,1_4_R,OA4,4,0,5,1


In [34]:
df_idseq.dtypes

sentenced    object
id_seq       object
dtype: object

In [36]:
df_idseq.head()

,sentenced,id_seq
0,Some models are managers and All models are cl...,1_0_R
1,No divers are carpenters and All linguists are...,1_1_R
2,All therapists are climbers and Some skaters a...,1_2_R
3,All bankers are golfers and All golfers are te...,1_3_R
4,Some boxers are not opticians and All boxers a...,1_4_R


In [38]:
df_nlp_final = df_nlp_result.merge(df_idseq, how="left", on ='id_seq')

In [41]:
df_nlp_final = df_nlp_final.drop(["Unnamed: 0","id_seq"], axis = 1)

In [44]:
df_nlp_final.to_csv("./data/nlp_result_final.csv", index=False)

# Analyse subset

In [ ]:
df_form = pd.read_csv("./data/form_data_final.csv")
df_nlp_result = pd.read_csv("./data/nlp_result_final.csv")

In [77]:
df_sub= df_nlp_result.merge(df_form, how="left", on='task_form')

In [78]:
df_sub.shape

(6506, 12)

In [80]:
df_sub.columns

Index(['task_form', 'best_bert_00_index', 'bert_00success',
       'best_mnli_00_index', 'mnli_00success', 'sentenced', 'has_conclusion',
       'index_conclusion', 'implicatur', 'mode', 'signe', 'absurde'],
      dtype='object')

In [82]:
df_sub.loc[ df_sub['has_conclusion']==True,['bert_00success', 'mnli_00success' ]].shape

(2709, 2)

In [81]:
df_sub.loc[ df_sub['has_conclusion']==True,['bert_00success', 'mnli_00success' ]].mean()

bert_00success    0.194168
mnli_00success    0.494647
dtype: float64

# Final data set

In [66]:
df_nlp_result = pd.read_csv("./data/nlp_result_final.csv")
df_form = pd.read_csv("./data/form_data_final.csv")
df_human = pd.read_csv("./data/human_data.csv")

In [67]:
df_final = df_human.merge(df_nlp_result, how="left", on ='sentenced')
df_final = df_final.merge(df_form, how="left", left_on='task_form_x', right_on = "task_form")

In [68]:
df_final = df_final.dropna()

In [69]:
import ast
df_final["human_success_bool"] = df_final[["human_response_index","index_conclusion"]].apply(lambda x: x[0] in ast.literal_eval(x[1]), axis = 1)
df_final["human_success"] = df_final["human_success_bool"].apply(lambda x: 1 if x == True else 0)

In [70]:
df_final = df_final[["human_response_index","human_success",'best_bert_00_index', 'bert_00success',
       'best_mnli_00_index', 'mnli_00success','task_form','implicatur', 'mode', 'signe', 'absurde']]

In [73]:
df_final.to_csv("./data/df_final.csv", index=False)

In [71]:
df_final.head()

,human_response_index,human_success,best_bert_00_index,bert_00success,best_mnli_00_index,mnli_00success,task_form,implicatur,mode,signe,absurde
0,2,1,2,1,2,1,IA4,False,particulier,positif,False
1,6,1,0,0,7,1,EA3,False,universel,negatif,False
2,3,1,3,1,2,1,AI2,False,particulier,positif,False
3,0,1,0,1,4,0,AA1,False,universel,positif,False
4,5,1,4,0,5,1,OA4,False,particulier,negatif,True


In [72]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4621 entries, 0 to 10952
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   human_response_index  4621 non-null   int64 
 1   human_success         4621 non-null   int64 
 2   best_bert_00_index    4621 non-null   int64 
 3   bert_00success        4621 non-null   int64 
 4   best_mnli_00_index    4621 non-null   int64 
 5   mnli_00success        4621 non-null   int64 
 6   task_form             4621 non-null   object
 7   implicatur            4621 non-null   object
 8   mode                  4621 non-null   object
 9   signe                 4621 non-null   object
 10  absurde               4621 non-null   object
dtypes: int64(6), object(5)
memory usage: 433.2+ KB


In [83]:
df_final["best_bert_00_index"].value_counts()

2    1768
0    1332
1     594
3     543
4     188
5     163
7      17
6      16
Name: best_bert_00_index, dtype: int64

In [84]:
df_final["best_mnli_00_index"].value_counts()

6    1182
5    1057
4     759
7     569
0     464
2     416
1      97
3      77
Name: best_mnli_00_index, dtype: int64

In [85]:
df_final["human_response_index"].value_counts()

8    705
6    702
7    611
5    577
4    573
2    543
3    468
0    267
1    175
Name: human_response_index, dtype: int64